In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "5d0d69d7",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Install dependencies\n",
    "!pip install neo4j sentence-transformers pinecone-client networkx matplotlib"
   ]
  },a
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "6d0c9e42",
   "metadata": {},
   "outputs": [],
   "source": [
    "from neo4j import GraphDatabase\n",
    "from sentence_transformers import SentenceTransformer\n",
    "import pinecone\n",
    "import networkx as nx\n",
    "import matplotlib.pyplot as plt"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "8f0a60b3",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Neo4j setup\n",
    "uri = \"bolt://localhost:7687\"\n",
    "driver = GraphDatabase.driver(uri, auth=(\"neo4j\", \"password\"))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "f2e08999",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Pinecone setup\n",
    "pinecone.init(api_key='your-api-key')\n",
    "index_name = 'neo4j-embedding-index'\n",
    "pinecone.create_index(index_name, dimension=384)\n",
    "index = pinecone.Index(index_name)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "f3ad1b0f",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Embedding model setup\n",
    "model = SentenceTransformer('all-MiniLM-L6-v2')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "c79d3de0",
   "metadata": {},
   "outputs": [],
   "source": [
    "def execute_query(query):\n",
    "    with driver.session() as session:\n",
    "        session.run(query)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "b9d1ff11",
   "metadata": {},
   "outputs": [],
   "source": [
    "def fetch_data():\n",
    "    query = \"MATCH (n)-[r]->(m) RETURN n, r, m\"\n",
    "    with driver.session() as session:\n",
    "        result = session.run(query)\n",
    "        return result.data()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "4b84d717",
   "metadata": {},
   "outputs": [],
   "source": [
    "def embed_data(data):\n",
    "    texts = [f\"{record['n']['name']} {record['r'].type} {record['m']['name']}\" for record in data]\n",
    "    embeddings = model.encode(texts)\n",
    "    return embeddings"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "d29d4e4b",
   "metadata": {},
   "outputs": [],
   "source": [
    "def upload_to_pinecone(embeddings, data):\n",
    "    for i, embedding in enumerate(embeddings):\n",
    "        index.upsert(vectors=[(str(i), embedding)], namespace='neo4j-data')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "8f0d31f0",
   "metadata": {},
   "outputs": [],
   "source": [
    "def nl_to_neoquery(nl_query):\n",
    "    if \"person\" in nl_query.lower():\n",
    "        return \"MATCH (n:Person) RETURN n\"\n",
    "    else:\n",
    "        return \"MATCH (n) RETURN n\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "d9e34cf7",
   "metadata": {},
   "outputs": [],
   "source": [
    "def fetch_neo_data(nl_query):\n",
    "    cypher_query = nl_to_neoquery(nl_query)\n",
    "    with driver.session() as session:\n",
    "        result = session.run(cypher_query)\n",
    "        return result.data()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "dd8d44d1",
   "metadata": {},
   "outputs": [],
   "source": [
    "def visualize_graph(data):\n",
    "    G = nx.Graph()\n",
    "    for record in data:\n",
    "        n = record['n']\n",
    "        m = record['m']\n",
    "        r = record['r']\n",
    "        G.add_node(n['name'], label=n.labels)\n",
    "        G.add_node(m['name'], label=m.labels)\n",
    "        G.add_edge(n['name'], m['name'], label=r.type)\n",
    "    \n",
    "    pos = nx.spring_layout(G)\n",
    "    nx.draw(G, pos, with_labels=True)\n",
    "    labels = nx.get_edge_attributes(G, 'label')\n",
    "    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "6ccf8e5e",
   "metadata": {},
   "outputs": [],
   "source": [
    "def fetch_combined_results(nl_query):\n",
    "    # Fetch from Neo4j\n",
    "    neo_data = fetch_neo_data(nl_query)\n",
    "    # Fetch from Pinecone\n",
    "    query_embedding = model.encode(nl_query)\n",
    "    pinecone_result = index.query(queries=[query_embedding], top_k=5, namespace='neo4j-data')\n",
    "    # Combine and display results\n",
    "    visualize_graph(neo_data)\n",
    "    return pinecone_result"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "f9d7e8e9",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Example workflow\n",
    "\n",
    "# Step 1: Populate Neo4j (example cypher query)\n",
    "cypher_query = \"\"\"\n",
    "CREATE (a:Person {name: 'Noble'})-[:KNOWS]->(b:Person {name: 'Biru'})\n",
    "\"\"\"\n",
    "execute_query(cypher_query)\n",
    "\n",
    "# Step 2: Fetch, Transform, and Embed Data\n",
    "data = fetch_data()\n",
    "embeddings = embed_data(data)\n",
    "\n",
    "# Step 3: Upload Embeddings to Pinecone\n",
    "upload_to_pinecone(embeddings, data)\n",
    "\n",
    "# Step 4: Query with Natural Language\n",
    "nl_query = \"Find all persons\"\n",
    "combined_results = fetch_combined_results(nl_query)\n",
    "print(combined_results)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
